In [ ]:

from kfp import dsl

@dsl.container_component
def check_nginx_version():
    return dsl.ContainerSpec(
        image='172.16.1.72:30002/library/nginx:1.25',
        # 注意：覆盖 command 会取代镜像原本的启动命令
        command=['nginx', '-v']
    )

@dsl.pipeline(name='check_nginx_pipeline')
def check_nginx_pipeline():
    check_nginx_version()

# Step 5: Compile
from kfp import compiler

compiler.Compiler().compile(check_nginx_pipeline, 'check_nginx_pipeline.yaml')

# Step 6: Run
from kfp.client import Client

client = Client(host='http://ml-pipeline.kubeflow:8888',
                namespace="kubeflow-user-example-com",
                existing_token="ACCESS_TOKEN",
                verify_ssl=False)
run = client.create_run_from_pipeline_package(
    'check_nginx_pipeline.yaml',
    enable_caching=False,  # 依然有效
    experiment_name='check_nginx_pipeline-test-experiment'
)
